In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import math
import numpy as np
from tkinter import *
import tensorflow as tf

In [2]:
root = Tk()

input_display_title = LabelFrame(root, text="NETWORK INPUTS", bg="#191919", background = "white", fg="black")
genome_stats_title = LabelFrame(root, text="GENOME STATS", bg="#191919", background = "white", fg="black")
log_display_title = LabelFrame(root, text="LOGS", bg="#191919", background = "white", fg="black")
game_stats_title = LabelFrame(root, text="GAME STATS", bg="#191919", background = "white", fg="black")

input_display_title.pack()
genome_stats_title.pack()
log_display_title.pack()
game_stats_title.pack()

input_display = Text(input_display_title, width=60, height=10, bg="#191919",
                     fg="black", background = "white", highlightbackground="#191919")
input_display.grid()
input_display.pack()

genome_stats = Text(genome_stats_title, width=60, height=6, bg="#191919",
                    fg="black",background = "white", highlightbackground="#191919")
genome_stats.grid()
genome_stats.pack()

log_display = Text(log_display_title, state="normal", wrap="none", width=60,
                   height=18, bg="#191919", fg="#7F1734", background = "white", highlightbackground="#191919")
log_display.grid()
log_display.pack()

game_stats = Text(game_stats_title, width=60, height=3, bg="#191919",
                  fg="black",background = "white", highlightbackground="#191919")
game_stats.grid()
game_stats.pack()

log_lines = 0
root.lift()
root.attributes('-topmost', True)
root.after_idle(root.attributes, '-topmost', False)

'after#0'

In [3]:
def writeToInput(array):
    input_display.delete(1.0, 'end')
    input_display.insert('end', "\t\t" + str(array[0][0]) + "\t" + str(array[0][4]) + "\t" +
                         str(array[0][8]) + "\t" + str(array[0][12]) + "\n\n\n")
    input_display.insert('end', "\t\t" + str(array[0][1]) + "\t" + str(array[0][5]) + "\t" +
                         str(array[0][9]) + "\t" + str(array[0][13]) + "\n\n\n")
    input_display.insert('end', "\t\t" + str(array[0][2]) + "\t" + str(array[0][6]) + "\t" +
                         str(array[0][10]) + "\t" + str(array[0][14]) + "\n\n\n")
    input_display.insert('end', "\t\t" + str(array[0][3]) + "\t" + str(array[0][7]) + "\t" +
                         str(array[0][11]) + "\t" + str(array[0][15]))
    root.update()


def writeToGenome_1(msg):
    genome_stats.delete(1.0, 'end')
    if msg == '[0]':
        genome_stats.insert('end', "Activation: UP\n")
    if msg == '[1]':
        genome_stats.insert('end', "Activation: DOWN\n")
    if msg == '[2]':
        genome_stats.insert('end', "Activation: LEFT\n")
    if msg == '[3]':
        genome_stats.insert('end', "Activation: RIGHT\n")
    genome_stats.insert('end', "Output: \n")
    genome_stats.tag_add("movehl", "1.11", "2.0")
    genome_stats.tag_config("movehl", foreground="#00008B")


def writeToGenome_2(msg):
    genome_stats.insert('end', msg+"\n")
    genome_stats.tag_add("outputhl", "2.8", "end")
    genome_stats.tag_config("outputhl", foreground="#00008B")
    root.update()


def writeToLog(msg):
    global log_lines
    log_lines += 1
    if log_lines == 18:
        log_display.delete(1.0, 2.0)
        log_lines -= 1
    log_display.insert('end', msg)
    log_display.insert('end', '\n')
    root.update()


def writeToGame(msg):
    game_stats.delete(1.0, 'end')
    game_stats.insert('end', "Fitness: " + str(msg) + "\n")
    game_stats.tag_add("fitnesshl", "1.8", "end")
    game_stats.tag_config("fitnesshl", foreground="#7F1734")


class Controller:
    def __init__(self):
        self.path_to_chromedriver = 'C:/Users/lenovo/Desktop/yapayZeka/2048/chromedriver.exe'
        self.browser = webdriver.Chrome(executable_path=self.path_to_chromedriver)
        self.url = 'https://gabrielecirulli.github.io/2048/'

    def loadPage(self):
        self.browser.get(self.url)
        time.sleep(2)
        element = self.browser.find_element_by_class_name('notice-close-button')
        element.click()
        self.browser.execute_script("document.body.style.zoom='75%'")

        main = self.browser.find_element_by_xpath('/html/body')
        main.send_keys(Keys.COMMAND, Keys.SUBTRACT)
        main.send_keys(Keys.COMMAND, Keys.SUBTRACT)
        time.sleep(1)

    def main(self):
        main = self.browser.find_element_by_xpath('/html/body')
        bot_01 = Bot("Genome 01")
        bot_02 = Bot("Genome 02")
        bot_03 = Bot("Genome 03")
        bot_04 = Bot("Genome 04")
        bot_05 = Bot("Genome 05")
        bot_06 = Bot("Genome 06")
        bot_07 = Bot("Genome 07")
        bot_08 = Bot("Genome 08")
        bot_09 = Bot("Genome 09")
        bot_10 = Bot("Genome 10")
        generation = [bot_01, bot_02, bot_03, bot_04, bot_05, bot_06, bot_07, bot_08, bot_09, bot_10 ]
        for gen in range(5):
            writeToLog('Executing generation '+str(gen))
            for x in range(10):
                game_state = True
                for y in range(10):
                    bot_x = generation[x]
                    score = self.browser.find_element_by_class_name('score-container').text
                    if '+' in score:
                        score = score[:score.index('+') - 1]
                    writeToGame(score)

                    tiles = self.loadTiles()
                    move = bot_x.run_neural_network(tiles)
                    if move == [0]:
                        main.send_keys(Keys.ARROW_UP)
                    if move == [1]:
                        main.send_keys(Keys.ARROW_DOWN)
                    if move == [2]:
                        main.send_keys(Keys.ARROW_LEFT)
                    if move == [3]:
                        main.send_keys(Keys.ARROW_RIGHT)
                    time.sleep(0.1)

                    msg = self.browser.find_element_by_class_name('game-message')
                    if 'Game over!' in msg.text:
                        game_state = False
                writeToLog(bot_x.get_name() + ' completed. Fitness: ' + score)
                bot_x.crossover_model(bot_10)
                bot_x.mutate_model()
                self.browser.execute_script("document.body.style.zoom='100%'")
                new_game = self.browser.find_element_by_class_name('restart-button')
                new_game.click()
                self.browser.execute_script("document.body.style.zoom='75%'")
                time.sleep(1)

        self.browser.close()

    def loadTiles(self):
        start = time.time()
        tiles = self.browser.find_elements_by_class_name('tile')
        tilearray = [[0, 0, 0, 0,
                      0, 0, 0, 0,
                      0, 0, 0, 0,
                      0, 0, 0, 0]]
        for tile in tiles:
            value = int(tile.text)
            string = tile.get_attribute('class')
            startpos = string.index('position')
            row = int(string[startpos + 9])
            col = int(string[startpos + 11])
            index = 4 * (row - 1) + (col - 1)

            if value > tilearray[0][index]:
                tilearray[0][index] = value
        writeToInput(tilearray)
        end = time.time()
        lag = float(end - start)
        return tilearray

In [4]:
class Bot:
    def __init__(self, msg):
        self.name = str(msg)
        self.n_inputs = 16
        self.n_nodes_hl1 = 10
        self.n_nodes_hl2 = 10
        self.n_nodes_hl3 = 10
        self.n_classes = 4
        self.mutate_chance = 0.5

        self.hidden_layer_1 = {'weights': np.random.random((self.n_inputs, self.n_nodes_hl1)),
                               'biases': np.random.random(self.n_nodes_hl1)}
        self.hidden_layer_2 = {'weights': np.random.random((self.n_nodes_hl1, self.n_nodes_hl2)),
                               'biases': np.random.random(self.n_nodes_hl2)}
        self.hidden_layer_3 = {'weights': np.random.random((self.n_nodes_hl2, self.n_nodes_hl3)),
                               'biases': np.random.random(self.n_nodes_hl3)}
        self.output_layer = {'weights': np.random.random((self.n_nodes_hl3, self.n_classes)),
                               'biases': np.random.random(self.n_classes)}

        writeToLog(self.name + " initialized")

    def get_name(self):
        return self.name

    def crossover_model(self, other):
        for value_a, value_b in zip(self.hidden_layer_1['weights'], other.hidden_layer_1['weights']):
            print("Value a:", value_a, "Value b:", value_b)


    def mutate_model(self):
        #Mutate layer 1
        for value in self.hidden_layer_1['weights']:
            for i in value:
                if self.mutate_chance >= np.random.random():
                    i = np.random.random()
        for value in self.hidden_layer_1['biases']:
                if self.mutate_chance >= np.random.random():
                    value = np.random.random()

        #Mutate layer 2
        for value in self.hidden_layer_2['weights']:
            for i in value:
                if self.mutate_chance >= np.random.random():
                    i = np.random.random()
        for value in self.hidden_layer_2['biases']:
            if self.mutate_chance >= np.random.random():
                value = np.random.random()

        #Mutate layer 3
        for value in self.hidden_layer_3['weights']:
            for i in value:
                if self.mutate_chance >= np.random.random():
                    i = np.random.random()
        for value in self.hidden_layer_3['biases']:
            if self.mutate_chance >= np.random.random():
                value = np.random.random()

        #Mutate output layer
        for value in self.output_layer['weights']:
            for i in value:
                if self.mutate_chance >= np.random.random():
                    i = np.random.random()
        for value in self.output_layer['biases']:
            if self.mutate_chance >= np.random.random():
                value = np.random.random()

    def neural_network_model(self, data):
        """Performs neural network operations"""
        l1 = np.add(np.dot(data, self.hidden_layer_1['weights']), self.hidden_layer_1['biases'])
        l1 = np.maximum(l1, 0)

        l2 = np.add(np.dot(l1, self.hidden_layer_2['weights']), self.hidden_layer_2['biases'])
        l2 = np.maximum(l2, 0)

        l3 = np.add(np.dot(l2, self.hidden_layer_3['weights']), self.hidden_layer_3['biases'])
        l3 = np.maximum(l3, 0)

        output = np.dot(l3, self.output_layer['weights']) + self.output_layer['biases']
        output = np.maximum(output, 0)
        return output

    def run_neural_network(self, x):
        """Returns output from neural network fed inputs"""
        start = time.time()
        keypress = self.neural_network_model(x)


        end = time.time()
        lag = float(end - start)
        writeToGenome_1(str(np.argmax(keypress, 1)))
        writeToGenome_2("UP: "+str(keypress[0][0]))
        writeToGenome_2("DOWN: "+str(keypress[0][1]))
        writeToGenome_2("LEFT: "+str(keypress[0][2]))
        writeToGenome_2("RIGHT: "+str(keypress[0][3]))
        return np.argmax(keypress, 1)

In [5]:
program = Controller()
program.loadPage()
program.main()

Value a: [0.14633083 0.65614871 0.57341415 0.29993797 0.72012531 0.40128224
 0.41186596 0.83674714 0.59942342 0.86928292] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.41063855 0.55253185 0.623512   0.64862619 0.35868189 0.64578386
 0.30116034 0.78270652 0.88282871 0.87733393] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.99848564 0.90040429 0.76721012 0.72524108 0.39507188 0.09811524
 0.55746251 0.54052261 0.04807531 0.90408411] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.15918476 0.5633661  0.58049933 0.05613756 0.25214986 0.59628304
 0.95007221 0.13108082 0.82166998 0.03248846] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.88540445 0.6

Value a: [0.03882959 0.89700968 0.64889626 0.47126404 0.11621679 0.87461379
 0.04076111 0.76289178 0.38825715 0.29584574] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.98735699 0.02364152 0.24460161 0.51850007 0.06882181 0.67207403
 0.81442422 0.87443281 0.02856644 0.5393267 ] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.67039048 0.28299999 0.6939037  0.30671682 0.30932388 0.42511601
 0.9916014  0.73080672 0.03281884 0.18447817] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.38886594 0.18262329 0.74564176 0.07548309 0.8007176  0.28340832
 0.55830762 0.74214762 0.90964918 0.01451811] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.45589792 0.4

Value a: [0.35968651 0.84247708 0.1255744  0.22459502 0.16226707 0.57652888
 0.12436275 0.55573663 0.74315897 0.09691278] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.54821074 0.5860985  0.92652526 0.36171341 0.35206662 0.78208904
 0.97512629 0.95503263 0.12030115 0.93263613] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.43844763 0.5366191  0.87984081 0.97844397 0.2137892  0.12987272
 0.9143195  0.14482381 0.55945219 0.48951793] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.37030563 0.52586534 0.88071967 0.74779525 0.74661417 0.71804405
 0.62002956 0.73841049 0.5891579  0.35407148] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.88525008 0.5

Value a: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.08539824 0.0

Value a: [0.78479579 0.17825586 0.64955547 0.72754121 0.47757961 0.55486125
 0.43668836 0.83873259 0.82274575 0.79124871] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.99452738 0.94088977 0.22222143 0.5880954  0.59976911 0.43817737
 0.08569463 0.13231914 0.99099845 0.46211965] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.90875674 0.85703092 0.87478453 0.53170008 0.42388203 0.96323489
 0.94701064 0.94659877 0.89832321 0.73269441] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.05033991 0.9755484  0.01406245 0.30784548 0.13893543 0.19023114
 0.08421612 0.10892627 0.67647522 0.54517758] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.39971731 0.6

...

Value a: [0.35968651 0.84247708 0.1255744  0.22459502 0.16226707 0.57652888
 0.12436275 0.55573663 0.74315897 0.09691278] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.54821074 0.5860985  0.92652526 0.36171341 0.35206662 0.78208904
 0.97512629 0.95503263 0.12030115 0.93263613] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.43844763 0.5366191  0.87984081 0.97844397 0.2137892  0.12987272
 0.9143195  0.14482381 0.55945219 0.48951793] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.37030563 0.52586534 0.88071967 0.74779525 0.74661417 0.71804405
 0.62002956 0.73841049 0.5891579  0.35407148] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.88525008 0.5

Value a: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.08539824 0.0

Value a: [0.78479579 0.17825586 0.64955547 0.72754121 0.47757961 0.55486125
 0.43668836 0.83873259 0.82274575 0.79124871] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.99452738 0.94088977 0.22222143 0.5880954  0.59976911 0.43817737
 0.08569463 0.13231914 0.99099845 0.46211965] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.90875674 0.85703092 0.87478453 0.53170008 0.42388203 0.96323489
 0.94701064 0.94659877 0.89832321 0.73269441] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.05033991 0.9755484  0.01406245 0.30784548 0.13893543 0.19023114
 0.08421612 0.10892627 0.67647522 0.54517758] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.39971731 0.6

Value a: [0.45417107 0.76436357 0.09613573 0.03216753 0.09312385 0.40385323
 0.36748061 0.71033379 0.66731406 0.59105598] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.38374241 0.99276529 0.68668489 0.74246433 0.06380609 0.85533629
 0.37338609 0.07152759 0.32532008 0.36021465] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.13942959 0.52391861 0.22942306 0.11097914 0.95383591 0.61781613
 0.91678584 0.78247108 0.52718651 0.68762631] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.42428331 0.37149204 0.57019878 0.97519988 0.025749   0.44614075
 0.77224359 0.02463968 0.94489088 0.9491768 ] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.73865675 0.1

Value a: [0.82626415 0.97972861 0.63156066 0.61532869 0.9564908  0.10924469
 0.01980732 0.39234322 0.82892867 0.19398142] Value b: [0.60473989 0.92749762 0.44848475 0.54892078 0.63971413 0.68721267
 0.69345289 0.05698005 0.87963907 0.13341104]
Value a: [0.64377783 0.27501715 0.81316429 0.84585693 0.13665029 0.888127
 0.48470873 0.10696027 0.95789508 0.81546975] Value b: [0.35250788 0.25037105 0.17330216 0.44454303 0.61116781 0.72457745
 0.43792379 0.01377856 0.75120837 0.40316215]
Value a: [0.01848836 0.4103884  0.18754673 0.6506009  0.38216549 0.66282509
 0.50071929 0.57098766 0.58347883 0.42893517] Value b: [0.47538989 0.70496467 0.86046004 0.54297678 0.14578548 0.38941138
 0.46388969 0.26549331 0.77192197 0.55204221]
Value a: [0.36810304 0.98102675 0.16897205 0.79055737 0.21559336 0.62385806
 0.36921482 0.44170743 0.55132824 0.65548792] Value b: [0.48818013 0.76824752 0.10683123 0.98490837 0.59918712 0.57339709
 0.91988027 0.43401057 0.60058786 0.5616316 ]
Value a: [0.50927669 0.480